In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bolobo,-2.1667,16.2333,72.25,88,100,3.36,CD,2022-02-25 01:51:41
1,1,Saint-Philippe,-21.3585,55.7679,74.79,86,100,6.46,RE,2022-02-25 01:51:41
2,2,Punta Arenas,-53.1500,-70.9167,55.51,71,0,12.66,CL,2022-02-25 01:47:49
3,3,Los Llanos De Aridane,28.6585,-17.9182,60.55,77,20,18.41,ES,2022-02-25 01:51:42
4,4,Cururupu,-1.8283,-44.8683,74.93,97,64,3.44,BR,2022-02-25 01:51:42


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# an array of length equal to the locations array length
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Los Banos,14.1667,121.1833,83.80,59,17,13.49,PH,2022-02-25 01:51:43
10,10,Rikitea,-23.1203,-134.9692,79.52,66,97,2.93,PF,2022-02-25 01:51:44
13,13,Puerto Ayora,-0.7393,-90.3518,75.34,86,86,7.05,EC,2022-02-25 01:51:45
17,17,Carnarvon,-24.8667,113.6333,82.47,69,9,18.41,AU,2022-02-25 01:51:46
18,18,Bandarbeyla,9.4942,50.8122,75.87,73,100,7.14,SO,2022-02-25 01:51:47
20,20,Atuona,-9.8000,-139.0333,78.84,76,13,19.84,PF,2022-02-25 01:51:47
22,22,Olinda,-8.0089,-34.8553,82.13,69,40,6.91,BR,2022-02-25 01:51:48
26,26,Busselton,-33.6500,115.3333,78.08,53,100,10.13,AU,2022-02-25 01:51:50
27,27,Popondetta,-8.7537,148.2534,89.62,60,100,5.30,PG,2022-02-25 01:51:50
32,32,Avarua,-21.2078,-159.7750,82.45,83,40,13.80,CK,2022-02-25 01:50:15


In [13]:
preferred_cities_df.count()

City_ID       128
City          128
Lat           128
Lng           128
Max Temp      128
Humidity      128
Cloudiness    128
Wind Speed    128
Country       128
Date          128
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Los Banos,PH,83.80,14.1667,121.1833,
10,Rikitea,PF,79.52,-23.1203,-134.9692,
13,Puerto Ayora,EC,75.34,-0.7393,-90.3518,
17,Carnarvon,AU,82.47,-24.8667,113.6333,
18,Bandarbeyla,SO,75.87,9.4942,50.8122,
20,Atuona,PF,78.84,-9.8000,-139.0333,
22,Olinda,BR,82.13,-8.0089,-34.8553,
26,Busselton,AU,78.08,-33.6500,115.3333,
27,Popondetta,PG,89.62,-8.7537,148.2534,
32,Avarua,CK,82.45,-21.2078,-159.7750,
